In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_mixup_cifar100.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def mixup_data(x, y, alpha=1.0, use_cuda=True):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(y_a, y_b, lam):
    return lambda criterion, pred: lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR100(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=100):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [5]:
checkpoint = None
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    alpha = 1.0
    
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha, use_cuda)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        
        loss_func = mixup_criterion(labels_a, labels_b, lam)
        loss = loss_func(criterion, outputs)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_mixup.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18_mixup_cifar100.txt", "a")
    f2.write('resnet_mixup_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
    
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18_mixup_cifar100.txt", "a")
        f3.write('resnet_mixup_cifar100测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_mixup_cifar100.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()

Epoch: [0][0/391]	Batch Time 3.953 (3.953)	Data Time 0.000 (0.000)	Loss 4.7913 (4.7913)	
Epoch: [0][20/391]	Batch Time 0.172 (0.355)	Data Time 0.000 (0.000)	Loss 4.4700 (4.5992)	
Epoch: [0][40/391]	Batch Time 0.187 (0.267)	Data Time 0.000 (0.000)	Loss 4.4797 (4.5278)	
Epoch: [0][60/391]	Batch Time 0.172 (0.237)	Data Time 0.000 (0.000)	Loss 4.3646 (4.4757)	
Epoch: [0][80/391]	Batch Time 0.172 (0.221)	Data Time 0.000 (0.000)	Loss 4.2895 (4.4326)	
Epoch: [0][100/391]	Batch Time 0.172 (0.212)	Data Time 0.000 (0.000)	Loss 4.0741 (4.3864)	
Epoch: [0][120/391]	Batch Time 0.172 (0.206)	Data Time 0.000 (0.000)	Loss 3.7770 (4.3570)	
Epoch: [0][140/391]	Batch Time 0.172 (0.202)	Data Time 0.000 (0.000)	Loss 3.8984 (4.3350)	
Epoch: [0][160/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 4.3453 (4.3055)	
Epoch: [0][180/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 3.7204 (4.2838)	
Epoch: [0][200/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 4.2791 (4.2642)	
Epoch

Epoch: [4][180/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 3.2424 (3.0896)	
Epoch: [4][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 3.3737 (3.0919)	
Epoch: [4][220/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 2.9822 (3.0820)	
Epoch: [4][240/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 2.4725 (3.1021)	
Epoch: [4][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 2.9089 (3.1002)	
Epoch: [4][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 3.5444 (3.0890)	
Epoch: [4][300/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 2.3287 (3.0853)	
Epoch: [4][320/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 3.1975 (3.0833)	
Epoch: [4][340/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 2.2450 (3.0815)	
Epoch: [4][360/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 2.5239 (3.0822)	
Epoch: [4][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 3.5074 (3.0771)	

Epoch: [8][360/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 3.2125 (2.7376)	
Epoch: [8][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 3.1165 (2.7360)	
tensor(0.2708)
测试分类准确率为：51.390%
Epoch: [9][0/391]	Batch Time 2.818 (2.818)	Data Time 0.000 (0.000)	Loss 1.7766 (1.7766)	
Epoch: [9][20/391]	Batch Time 0.172 (0.303)	Data Time 0.000 (0.000)	Loss 3.3010 (2.7174)	
Epoch: [9][40/391]	Batch Time 0.172 (0.242)	Data Time 0.000 (0.000)	Loss 2.5182 (2.8173)	
Epoch: [9][60/391]	Batch Time 0.187 (0.221)	Data Time 0.000 (0.000)	Loss 3.1573 (2.7494)	
Epoch: [9][80/391]	Batch Time 0.172 (0.210)	Data Time 0.000 (0.000)	Loss 3.4879 (2.7650)	
Epoch: [9][100/391]	Batch Time 0.172 (0.204)	Data Time 0.000 (0.000)	Loss 2.1806 (2.7430)	
Epoch: [9][120/391]	Batch Time 0.172 (0.200)	Data Time 0.000 (0.000)	Loss 3.0489 (2.7335)	
Epoch: [9][140/391]	Batch Time 0.172 (0.197)	Data Time 0.000 (0.000)	Loss 2.9436 (2.7477)	
Epoch: [9][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.

Epoch: [13][120/391]	Batch Time 0.180 (0.199)	Data Time 0.000 (0.000)	Loss 1.9412 (2.5752)	
Epoch: [13][140/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 2.9317 (2.5593)	
Epoch: [13][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 2.8396 (2.5418)	
Epoch: [13][180/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 3.3061 (2.5457)	
Epoch: [13][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 2.1317 (2.5588)	
Epoch: [13][220/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 3.0918 (2.5384)	
Epoch: [13][240/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 2.8547 (2.5444)	
Epoch: [13][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 2.2190 (2.5491)	
Epoch: [13][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 1.6607 (2.5330)	
Epoch: [13][300/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 2.9893 (2.5469)	
Epoch: [13][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 3.082

Epoch: [17][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 2.3699 (2.4781)	
Epoch: [17][300/391]	Batch Time 0.201 (0.187)	Data Time 0.000 (0.000)	Loss 2.2183 (2.4690)	
Epoch: [17][320/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 2.8728 (2.4580)	
Epoch: [17][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 2.7800 (2.4501)	
Epoch: [17][360/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 1.7464 (2.4592)	
Epoch: [17][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 3.2295 (2.4750)	
tensor(0.3032)
测试分类准确率为：60.330%
Epoch: [18][0/391]	Batch Time 2.744 (2.744)	Data Time 0.000 (0.000)	Loss 2.1265 (2.1265)	
Epoch: [18][20/391]	Batch Time 0.187 (0.300)	Data Time 0.000 (0.000)	Loss 2.5292 (2.4991)	
Epoch: [18][40/391]	Batch Time 0.187 (0.240)	Data Time 0.000 (0.000)	Loss 3.1456 (2.4859)	
Epoch: [18][60/391]	Batch Time 0.172 (0.220)	Data Time 0.000 (0.000)	Loss 2.7572 (2.5125)	
Epoch: [18][80/391]	Batch Time 0.187 (0.210)	Data Tim

...

Epoch: [179][260/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 2.2011 (1.5606)	
Epoch: [179][280/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 1.8954 (1.5587)	
Epoch: [179][300/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 2.1687 (1.5537)	
Epoch: [179][320/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.4606 (1.5333)	
Epoch: [179][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 2.2294 (1.5280)	
Epoch: [179][360/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 1.2638 (1.5359)	
Epoch: [179][380/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.1479 (1.5254)	
tensor(0.4726)
测试分类准确率为：77.430%
Epoch: [180][0/391]	Batch Time 2.809 (2.809)	Data Time 0.000 (0.000)	Loss 2.2488 (2.2488)	
Epoch: [180][20/391]	Batch Time 0.172 (0.304)	Data Time 0.000 (0.000)	Loss 1.8627 (1.4206)	
Epoch: [180][40/391]	Batch Time 0.172 (0.242)	Data Time 0.000 (0.000)	Loss 0.6175 (1.4187)	
Epoch: [180][60/391]	Batch Time 0.187 (0.2

tensor(0.4983)
测试分类准确率为：77.770%
Epoch: [184][0/391]	Batch Time 2.725 (2.725)	Data Time 0.000 (0.000)	Loss 1.5824 (1.5824)	
Epoch: [184][20/391]	Batch Time 0.172 (0.299)	Data Time 0.000 (0.000)	Loss 2.3600 (1.5649)	
Epoch: [184][40/391]	Batch Time 0.172 (0.240)	Data Time 0.000 (0.000)	Loss 1.4712 (1.6303)	
Epoch: [184][60/391]	Batch Time 0.172 (0.220)	Data Time 0.000 (0.000)	Loss 2.1975 (1.5820)	
Epoch: [184][80/391]	Batch Time 0.172 (0.209)	Data Time 0.000 (0.000)	Loss 1.7313 (1.6010)	
Epoch: [184][100/391]	Batch Time 0.172 (0.203)	Data Time 0.000 (0.000)	Loss 1.0851 (1.5493)	
Epoch: [184][120/391]	Batch Time 0.172 (0.199)	Data Time 0.000 (0.000)	Loss 2.3564 (1.5445)	
Epoch: [184][140/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 1.7123 (1.5717)	
Epoch: [184][160/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 2.1287 (1.6156)	
Epoch: [184][180/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 1.9752 (1.6156)	
Epoch: [184][200/391]	Batch Time 0.172 (0.19

Epoch: [188][140/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 2.2567 (1.6002)	
Epoch: [188][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 1.7826 (1.6186)	
Epoch: [188][180/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 0.2888 (1.5635)	
Epoch: [188][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 2.1752 (1.5537)	
Epoch: [188][220/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 2.2788 (1.5591)	
Epoch: [188][240/391]	Batch Time 0.187 (0.189)	Data Time 0.000 (0.000)	Loss 2.2226 (1.5700)	
Epoch: [188][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 1.9485 (1.5785)	
Epoch: [188][280/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.1665 (1.5538)	
Epoch: [188][300/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.8493 (1.5403)	
Epoch: [188][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 1.7607 (1.5340)	
Epoch: [188][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)

Epoch: [192][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 1.8903 (1.4925)	
Epoch: [192][300/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.3024 (1.4872)	
Epoch: [192][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 2.2815 (1.4876)	
Epoch: [192][340/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 2.2147 (1.4862)	
Epoch: [192][360/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 1.3890 (1.4708)	
Epoch: [192][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 2.3100 (1.4739)	
tensor(0.4859)
测试分类准确率为：77.810%
Epoch: [193][0/391]	Batch Time 2.741 (2.741)	Data Time 0.000 (0.000)	Loss 1.9575 (1.9575)	
Epoch: [193][20/391]	Batch Time 0.187 (0.300)	Data Time 0.000 (0.000)	Loss 2.3141 (1.3851)	
Epoch: [193][40/391]	Batch Time 0.172 (0.240)	Data Time 0.000 (0.000)	Loss 2.2867 (1.4264)	
Epoch: [193][60/391]	Batch Time 0.172 (0.220)	Data Time 0.000 (0.000)	Loss 2.1793 (1.4565)	
Epoch: [193][80/391]	Batch Time 0.172 (0.21

Epoch: [197][20/391]	Batch Time 0.172 (0.300)	Data Time 0.000 (0.000)	Loss 2.2631 (1.6721)	
Epoch: [197][40/391]	Batch Time 0.172 (0.241)	Data Time 0.000 (0.000)	Loss 2.3618 (1.5970)	
Epoch: [197][60/391]	Batch Time 0.187 (0.220)	Data Time 0.000 (0.000)	Loss 0.6043 (1.5412)	
Epoch: [197][80/391]	Batch Time 0.187 (0.210)	Data Time 0.000 (0.000)	Loss 1.7741 (1.5522)	
Epoch: [197][100/391]	Batch Time 0.187 (0.204)	Data Time 0.000 (0.000)	Loss 1.5953 (1.5412)	
Epoch: [197][120/391]	Batch Time 0.187 (0.199)	Data Time 0.000 (0.000)	Loss 2.2518 (1.5477)	
Epoch: [197][140/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 2.1132 (1.5739)	
Epoch: [197][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 2.0310 (1.5432)	
Epoch: [197][180/391]	Batch Time 0.172 (0.193)	Data Time 0.000 (0.000)	Loss 1.4591 (1.5555)	
Epoch: [197][200/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 1.4453 (1.5344)	
Epoch: [197][220/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Los

In [7]:
use_cuda

True